In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
glob.glob('./Second pass model outputs/*.txt')

['./Second pass model outputs\\ByT5 Console Output.txt',
 './Second pass model outputs\\byt5 inference clean data.txt',
 './Second pass model outputs\\byt5 inference data.txt',
 './Second pass model outputs\\byt5 multi iteration data.txt',
 './Second pass model outputs\\T5 Console Output.txt',
 './Second pass model outputs\\t5 full dateaset word segment.txt',
 './Second pass model outputs\\t5 inference data.txt',
 './Second pass model outputs\\t5 multi iteration 70 percentage data trained on.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model_2.txt',
 './Second pass model outputs\\t5 multi iteration data word segmented.txt',
 './Second pass model outputs\\t5 multi iteration data.txt',
 './Second pass model outputs\\t5 multi iteration wod segmented 2 epochs.txt',
 './Second pass model outputs\\t5 training output.txt',
 './Second pass model outputs\\t5_small on all dataset 

In [8]:
txt_path = "./Second pass model outputs\\t5_small_new_embedding.txt"

lines = open(txt_path, 'r').readlines()

In [11]:
for line in lines:
    if '2023' in line:
        print(line)

08/01/2023

08/02/2023

08/03/2023

08/04/2023

08/05/2023

08/06/2023

An error has occured in 08/06/2023.

08/07/2023

08/08/2023

08/09/2023

08/10/2023

An error has occured in 08/10/2023.

08/11/2023

An error has occured in 08/11/2023.

08/12/2023

An error has occured in 08/12/2023.

08/13/2023

An error has occured in 08/13/2023.

08/14/2023

08/15/2023

An error has occured in 08/15/2023.

08/16/2023

An error has occured in 08/16/2023.

08/17/2023

An error has occured in 08/17/2023.

08/18/2023

An error has occured in 08/18/2023.

08/19/2023

An error has occured in 08/19/2023.

08/20/2023

08/21/2023

08/22/2023

08/23/2023

An error has occured in 08/23/2023.

08/24/2023

An error has occured in 08/24/2023.

08/25/2023

An error has occured in 08/25/2023.

08/26/2023

An error has occured in 08/26/2023.

08/27/2023

An error has occured in 08/27/2023.

08/28/2023

08/29/2023

An error has occured in 08/29/2023.

08/30/2023

An error has occured in 08/30/2023.

08/31/2023
